# 원, 삼각형, 사각형 구분하기
## 3. Data Augmentation을 사용한 CNN 모델

#### https://tykimos.github.io/2017/06/10/CNN_Data_Augmentation/

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# seed 값 설정
seed = 2020
np.random.seed(seed)
tf.random.set_seed(seed)

### 데이터 부풀리기(Augmentation)

In [3]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img

In [4]:
# 데이터 부풀리기를 적용한 훈련 데이터 제너레이터
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=15,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.5,
                                   zoom_range=[0.8, 2.0],
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

In [5]:
img = load_img('hard_handwriting_shape/train/triangle/triangle001.png')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0

# 무한 for loop이니 주의!!!
for batch in train_datagen.flow(x, batch_size=1, save_to_dir='preview',
                                save_prefix='tri', save_format='png'):
    i += 1
    if i > 30: 
        break
# preview파일에 30개 생성

### 데이터셋 생성하기

In [6]:
train_generator = train_datagen.flow_from_directory(
        'hard_handwriting_shape/train',
        target_size=(24, 24),
        batch_size=3,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'hard_handwriting_shape/test',
        target_size=(24, 24),    
        batch_size=3,
        class_mode='categorical')

# 훈련할 때는 데이터 부풀리기를 하지만 테스트를 할 때는 원본데이터 그대로

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


### 모델 구성하기

In [7]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu', input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               819328    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
________________________________________________

### 모델 학습과정 설정하기

In [8]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

### 모델 학습시키기
- 첫번째 인자 : 훈련데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 train_generator으로 지정.
- steps_per_epoch : 한 epoch에 사용한 스텝 수를 지정. 총 45개의 훈련 샘플이 있고 배치사이즈가 3이므로 15 스텝으로 지정.
- epochs : 전체 훈련 데이터셋에 대해 학습 반복 횟수를 지정. 200번을 반복적으로 학습.
- validation_data : 검증데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 validation_generator으로 지정.
- validation_steps : 한 epoch 종료 시 마다 검증할 때 사용되는 검증 스텝 수를 지정. 홍 15개의 검증 샘플이 있고 배치사이즈가 3이므로 5 스텝으로 지정.

In [9]:
model.fit_generator(
        train_generator,
        steps_per_epoch=15,   # 원래는*100
        epochs=200,
        validation_data=test_generator,
        validation_steps=5)

Epoch 1/200
15/15 [==============================] - 0s 18ms/step - loss: 1.1867 - accuracy: 0.4000 - val_loss: 1.1025 - val_accuracy: 0.3333
Epoch 2/200
15/15 [==============================] - 0s 11ms/step - loss: 1.0913 - accuracy: 0.3333 - val_loss: 1.1486 - val_accuracy: 0.2667
Epoch 3/200
15/15 [==============================] - 0s 10ms/step - loss: 0.9780 - accuracy: 0.6444 - val_loss: 1.1727 - val_accuracy: 0.4667
Epoch 4/200
15/15 [==============================] - 0s 10ms/step - loss: 0.7895 - accuracy: 0.6667 - val_loss: 0.9612 - val_accuracy: 0.3333
Epoch 5/200
15/15 [==============================] - 0s 11ms/step - loss: 0.8033 - accuracy: 0.6444 - val_loss: 1.6188 - val_accuracy: 0.5333
Epoch 6/200
15/15 [==============================] - 0s 11ms/step - loss: 0.6876 - accuracy: 0.7556 - val_loss: 2.2473 - val_accuracy: 0.6667
Epoch 7/200
15/15 [==============================] - 0s 11ms/step - loss: 0.7888 - accuracy: 0.6667 - val_loss: 0.6287 - val_accuracy: 0.4667
Epoch 

15/15 [==============================] - 0s 10ms/step - loss: 0.0300 - accuracy: 0.9778 - val_loss: 0.7126 - val_accuracy: 0.7333
Epoch 59/200
15/15 [==============================] - 0s 10ms/step - loss: 0.0868 - accuracy: 0.9556 - val_loss: 0.5976 - val_accuracy: 0.7333
Epoch 60/200
15/15 [==============================] - 0s 10ms/step - loss: 0.1285 - accuracy: 0.9556 - val_loss: 3.7744 - val_accuracy: 0.7333
Epoch 61/200
15/15 [==============================] - 0s 10ms/step - loss: 0.0258 - accuracy: 1.0000 - val_loss: 0.0222 - val_accuracy: 0.6667
Epoch 62/200
15/15 [==============================] - 0s 10ms/step - loss: 0.0477 - accuracy: 0.9778 - val_loss: 0.2643 - val_accuracy: 0.7333
Epoch 63/200
15/15 [==============================] - 0s 10ms/step - loss: 0.0552 - accuracy: 0.9556 - val_loss: 0.1547 - val_accuracy: 0.7333
Epoch 64/200
15/15 [==============================] - 0s 10ms/step - loss: 0.0798 - accuracy: 0.9778 - val_loss: 4.8956 - val_accuracy: 0.7333
Epoch 65/200

15/15 [==============================] - 0s 10ms/step - loss: 0.1380 - accuracy: 0.9778 - val_loss: 2.8256 - val_accuracy: 0.8667
Epoch 116/200
15/15 [==============================] - 0s 10ms/step - loss: 0.0861 - accuracy: 0.9556 - val_loss: 0.1655 - val_accuracy: 0.7333
Epoch 117/200
15/15 [==============================] - 0s 10ms/step - loss: 0.1018 - accuracy: 0.9556 - val_loss: 0.5920 - val_accuracy: 0.6000
Epoch 118/200
15/15 [==============================] - 0s 10ms/step - loss: 0.0482 - accuracy: 1.0000 - val_loss: 0.9337 - val_accuracy: 0.5333
Epoch 119/200
15/15 [==============================] - 0s 10ms/step - loss: 0.1187 - accuracy: 0.9556 - val_loss: 5.4933 - val_accuracy: 0.8000
Epoch 120/200
15/15 [==============================] - 0s 10ms/step - loss: 0.0142 - accuracy: 1.0000 - val_loss: 1.1785e-04 - val_accuracy: 0.7333
Epoch 121/200
15/15 [==============================] - 0s 10ms/step - loss: 0.0100 - accuracy: 1.0000 - val_loss: 0.1844 - val_accuracy: 0.7333
Ep

15/15 [==============================] - 0s 10ms/step - loss: 0.0087 - accuracy: 1.0000 - val_loss: 0.0700 - val_accuracy: 0.7333
Epoch 172/200
15/15 [==============================] - 0s 10ms/step - loss: 0.1072 - accuracy: 0.9778 - val_loss: 3.0509 - val_accuracy: 0.8000
Epoch 173/200
15/15 [==============================] - 0s 10ms/step - loss: 0.1383 - accuracy: 0.9556 - val_loss: 6.4933 - val_accuracy: 0.6667
Epoch 174/200
15/15 [==============================] - 0s 10ms/step - loss: 0.0739 - accuracy: 0.9556 - val_loss: 0.6972 - val_accuracy: 0.7333
Epoch 175/200
15/15 [==============================] - 0s 10ms/step - loss: 0.0263 - accuracy: 1.0000 - val_loss: 3.5245 - val_accuracy: 0.8000
Epoch 176/200
15/15 [==============================] - 0s 10ms/step - loss: 0.0309 - accuracy: 0.9778 - val_loss: 1.7555e-04 - val_accuracy: 0.8000
Epoch 177/200
15/15 [==============================] - 0s 10ms/step - loss: 0.0221 - accuracy: 0.9778 - val_loss: 2.8692 - val_accuracy: 0.7333
Ep

### 모델 평가하기

In [10]:
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
accuracy: 73.33%


### 모델 사용하기

In [11]:
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

-- Predict --
{'circle': 0, 'rectangle': 1, 'triangle': 2}
[[0.001 0.996 0.003]
 [1.000 0.000 0.000]
 [0.000 1.000 0.000]
 [0.000 0.000 1.000]
 [0.000 0.000 1.000]
 [0.995 0.000 0.005]
 [0.000 0.000 1.000]
 [0.001 0.115 0.884]
 [0.000 0.000 1.000]
 [0.833 0.001 0.167]
 [0.004 0.888 0.108]
 [0.000 0.000 1.000]
 [0.000 0.000 1.000]
 [0.000 0.000 1.000]
 [0.000 0.000 1.000]]


In [12]:
for i in range(len(test_generator.labels)):
    print(test_generator.filenames[i], test_generator.labels[i], np.argmax(output[i]))

circle\circle021.png 0 1
circle\circle022.png 0 0
circle\circle023.png 0 1
circle\circle024.png 0 2
circle\circle025.png 0 2
rectangle\rectangle021.png 1 0
rectangle\rectangle022.png 1 2
rectangle\rectangle023.png 1 2
rectangle\rectangle024.png 1 2
rectangle\rectangle025.png 1 0
triangle\triangle021.png 2 1
triangle\triangle022.png 2 2
triangle\triangle023.png 2 2
triangle\triangle024.png 2 2
triangle\triangle025.png 2 2
